In [1]:
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd
import label_improve

In [5]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [13]:
# Loading the data 
dataset_name = "chemprot"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

# Sample a dev set to help seed ideas for LFs
dev_df = train_df.sample(250, random_state=123)


In [20]:

# show the first row of the dataframe
train_df.head(1)

,text,labels,entity1,entity2,span1,span2,weak_labels
10305,"Selective inhibition of PDE5 is a rational therapeutic approach in ED, as proved by the clinical success of sildenafil.",3,PDE5,sildenafil,"[24, 28]","[108, 118]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]"


## Main Work

In [11]:
# Cell to inspect the dev set for a given label 
label = 'NOT' # Change this variable
pd.set_option('display.max_colwidth', 1000)
dev_df[dev_df['labels'] == int(label_to_idx[label])]

,text,labels
9357,Administration of DMI for 14 or 21 days did not further reduce the number of beta-adrenoceptors.,9
8191,"In cultured cardiomyocytes, cardiomyocyte hypertrophy induced by the α(1)-agonist phenylephrine was inhibited by the overexpression of SIRT1 as well as resveratrol, both of which down-regulated p300 protein levels but not p300 mRNA levels.",9
5813,"In six patients with Cushing's disease and one patient with secondary adrenal insufficiency due to hypothalamic failure, neither basal ACTH and cortisol levels nor CRH-stimulated levels were influenced by loperamide.",9
14459,"Unlike the majority of G protein-coupled receptors, the prostaglandin E(2) (PGE(2)) E-prostanoid 3 (EP3) receptor binds agonist with high affinity that is insensitive to the presence of guanosine 5[prime]-O-(3-thio)triphosphate (GTPγS).",9
12124,"In addition, the stereospecificity required at opioid receptors appears to be retained at the nociceptin receptor, since (+)-quadazocine is inactive at both receptors.",9
12862,"The purified Atp8a1 is inactive in detergent micelles or in micelles containing phosphatidylcholine, phosphatidic acid, or phosphatidylinositol, is minimally activated by phosphatidylglycerol or phosphatidylethanolamine (PE), and is maximally activated by PS.",9
1517,"However, L-nor-okadaone (0.001 pg/mouse-1 ng/mouse, i.c.v. ), an analogue of okadaic acid lacking activity against protein phosphatases, did not affect the antinociceptive effect of morphine.",9
10347,GW660511X and omapatrilat increased the production of both BrBK1-8 and Br-Phe5 but not that of BrBK4-8 and BrBK2-8.,9


In [228]:
# Cell to inspect dev set for a given keyword (to inspect conflicts). Note: this doesn't work with "+" 
keyword = 'iPad' # Change this word
pd.set_option('display.max_colwidth', 1000)
dev_df[dev_df['text'].str.contains(keyword)]

,text,labels
42750,"Lenovo IdeaTab K3 Lynx 11.6-Inch 64 GB Tablet Smudgy screen brings it shy of 5 stars.... I'm only going to review the tablet, and not Windows 8, Office, etc...the os and program behavior on the tablet is identical to ANY machine running full Windows 8, so no need for ANOTHER Windows 8 review!<br />.<br />So far I'm not bothered by any so-called &#34;flex&#34; others are talking about. Yes, it's plastic and therefore plasticity, but it feels firm enough to me. I'm willing to sacrifice ultra firmness (like on the iPad) for something as light as this.<br /><br />The screen is bigger than the biggest iPad, which my wife has, but i can hold and read a digital magazine on the Lynx in portrait mode with one hand and not feel the strain like I do with the iPad.<br /><br />That being said, the screen IS an awkward size: in portrait mode, the magazines don't take up the full screen, leaving empty black space at the top and bottom.<br /><br />I put in a 64gb micro SDXC card the other day, and...",20


## Evaluation

In [235]:
L_train = apply_LFs(lfs, train_df)
L_valid = apply_LFs(lfs, valid_df)
L_test = apply_LFs(lfs, test_df)

print("Train Coverage:", calc_coverage(L_train))
print("Valid Coverage:", calc_coverage(L_valid))
print("Test Coverage:", calc_coverage(L_test))

lf_analysis = LFAnalysis(L=L_valid, lfs=lfs).lf_summary()

# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

100%|██████████| 17402/17402 [00:22<00:00, 756.63it/s]


Train Coverage: 0.6395003832115403
Valid Coverage: 0.6396210163652024
Test Coverage: 0.6375703942075623


,j,Polarity,Coverage,Overlaps,Conflicts,Conflict Ratio
lf_blouse,0,[0],0.001206,0.000861,0.000517,0.428571
lf_shirt,1,[0],0.009991,0.006202,0.005857,0.586207
lf_scarf,2,[0],0.001378,0.000689,0.000689,0.500000
lf_jacket,3,[0],0.004479,0.002239,0.002239,0.500000
lf_underwear,4,[0],0.001206,0.000517,0.000345,0.285714
...,...,...,...,...,...,...
lf_bracelet+watch,225,[29],0.003101,0.003101,0.003101,1.000000
lf_electronic+case,226,[30],0.001206,0.001034,0.001034,0.857143
lf_ipod,227,[30],0.012231,0.008786,0.007407,0.605634
lf_ipad,228,[30],0.008613,0.006546,0.005168,0.600000


In [236]:
# List LFs for which 'Conflict Ratio' is above some threshold (helpful for debugging)
lf_analysis[lf_analysis['Conflict Ratio'] > 0.8]['Conflict Ratio'].sort_values(ascending=False)

lf_poncho              1.000000
lf_portable battery    1.000000
lf_bracelet+watch      1.000000
lf_card game           1.000000
lf_tent+daughter       1.000000
lf_car+1:              1.000000
lf_athletic            1.000000
lf_t-shirt             1.000000
lf_video+conversion    1.000000
lf_basketball+shoe     1.000000
lf_cage                1.000000
lf_laptop stand        1.000000
lf_folders             1.000000
lf_headphone+studio    1.000000
lf_pop+classic         1.000000
lf_toners              1.000000
lf_acne treatment      1.000000
lf_skin care           1.000000
lf_waist trimmer       1.000000
lf_nail+paint          1.000000
lf_gummi               1.000000
lf_laptop+battery      0.909091
lf_punch               0.875000
lf_air+drain           0.875000
lf_electronic+case     0.857143
lf_tent+son            0.842105
lf_laptop+case         0.833333
Name: Conflict Ratio, dtype: float64

In [237]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(31)
preds_valid = majority_model.predict(L=L_valid)
(preds_valid[preds_valid != -1] == valid_df[preds_valid != -1].labels.values).mean()

0.6649519890260631

In [238]:
json.dump(keywords, open("amazon_LFs_v1.json", "w"), indent=8)

In [ ]:
# Replace the LFs for a given dataset (in wrench format)
# dataset_name = "dbpedia"

# train_json = json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r"))
# for idx in train_json:
#     train_json[idx]['weak_labels'] = [int(i) for i in list(L_train[int(idx)])]
    
# valid_json = json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r"))
# for idx in valid_json:
#     valid_json[idx]['weak_labels'] = [int(i) for i in list(L_valid[int(idx)])]
    
# test_json = json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r"))
# for idx in test_json:
#     test_json[idx]['weak_labels'] = [int(i) for i in list(L_test[int(idx)])]

# json.dump(train_json, open(f"../weak_datasets/{dataset_name}/train.json", 'w'), indent=4)
# json.dump(valid_json, open(f"../weak_datasets/{dataset_name}/valid.json", 'w'), indent=4)
# json.dump(test_json, open(f"../weak_datasets/{dataset_name}/test.json", 'w'), indent=4)